In [2]:
import os
import re
import pickle

import pandas as pd
import matplotlib.pyplot as plt

from src.env import PROJECT_DIR

In [11]:
with open(PROJECT_DIR / 'data' / 'interim' / 'all_articles_1.pkl', 'rb') as fp:
    dfs = pickle.load(fp)

In [12]:
dfs.keys()

dict_keys(['경향신문', '동아일보', '조선일보', '한겨레'])

In [5]:
import torch

In [6]:
from kobert_transformers import get_kobert_model, get_distilkobert_model
model = get_kobert_model()
model.eval()
input_ids = torch.LongTensor([[31, 51, 99], [15, 5, 0]])
attention_mask = torch.LongTensor([[1, 1, 1], [1, 1, 0]])
token_type_ids = torch.LongTensor([[0, 0, 1], [0, 1, 0]])
sequence_output, pooled_output = model(input_ids, attention_mask, token_type_ids)
sequence_output[0]

tensor([[-0.2461,  0.2428,  0.2590,  ..., -0.4861, -0.0731,  0.0756],
        [-0.2478,  0.2420,  0.2552,  ..., -0.4877, -0.0727,  0.0754],
        [-0.2472,  0.2420,  0.2561,  ..., -0.4874, -0.0733,  0.0765]],
       grad_fn=<SelectBackward>)

In [7]:
from kobert_transformers import get_tokenizer

In [8]:
tokenizer = get_tokenizer()

In [9]:
tokenizer.tokenize("[CLS] 한국어 모델을 공유합니다. [SEP]")

['[CLS]', '▁한국', '어', '▁모델', '을', '▁공유', '합니다', '.', '[SEP]']

In [13]:
df = dfs['경향신문']
df.head()

,title,summary,text,extracted_category,sentences
datetime,,,,,
2017-01-03 18:00:00,"대선주자 인터뷰②이재명 ""나? 좋은 사람 아니라 부패척결 머슴..문재인은 점잖은 성군""",NaN,\n\n촛불정국을 거치면서 야권의 유력 대선주자로 발돋움한 더불어민주당 이재명 성남...,None,[\n\n촛불정국을 거치면서 야권의 유력 대선주자로 발돋움한 더불어민주당 이재명 성...
2017-01-03 22:29:00,"[2017 대선의 꿈]②이재명 성남시장 ""혁명적 변화가 필요한 때..거친 야전형 장...",NaN,\n\n촛불정국을 거치면서 야권의 유력 대선주자로 발돋움한 더불어민주당 이재명 성남...,2017 대선의 꿈,[\n\n촛불정국을 거치면서 야권의 유력 대선주자로 발돋움한 더불어민주당 이재명 성...
2017-01-04 14:57:00,아인슈타인은 옳았다..왜 노동시간을 줄여야 하는가?,NaN,"인공지능과 로봇, 3D 프린팅이 주도하는 4차 산업혁명이 일자리 킬러가 될 수 있...",None,"[ 인공지능과 로봇, 3D 프린팅이 주도하는 4차 산업혁명이 일자리 킬러가 될 수 ..."
2017-01-06 19:33:00,[책과 삶]기본소득 보장은 재앙이 아니라 미래의 대안,NaN,ㆍ기본소득이 세상을 바꾼다ㆍ오준호 지음 |개마고원 | 232쪽 | 1만4000원\...,책과 삶,[ ㆍ기본소득이 세상을 바꾼다ㆍ오준호 지음 |개마고원 | 232쪽 | 1만4000원...
2017-01-08 21:04:00,[사설]소득 불평등 해소 없으면 개혁 아니다,NaN,ㆍ구체제 청산을 위한 7대 과제\n\n덴마크에서 붙잡힌 최순실씨 딸 정유라씨는 값...,사설,[ ㆍ구체제 청산을 위한 7대 과제\n\n덴마크에서 붙잡힌 최순실씨 딸 정유라씨는 ...


In [18]:
res = tokenizer.tokenize("[CLS]" + " [SEP] ".join([s.strip() for s in df.iloc[0].sentences]))

In [19]:
res

['[CLS]',
 '▁',
 '촛',
 '불',
 '정',
 '국',
 '을',
 '▁거치',
 '면서',
 '▁야',
 '권',
 '의',
 '▁유력',
 '▁대선',
 '주',
 '자로',
 '▁발',
 '돋',
 '움',
 '한',
 '▁더불어',
 '민주당',
 '▁이재',
 '명',
 '▁성남',
 '시장',
 '▁(',
 '53',
 ')',
 '▁',
 '은',
 '▁3',
 '일',
 '▁남',
 '들은',
 '▁다',
 '▁성장',
 ',',
 '▁성장',
 '▁하는데',
 ',',
 '▁그',
 '거',
 '야',
 '말',
 '로',
 '▁포',
 '퓰',
 '리',
 '즘',
 '이라며',
 '▁경제적',
 '▁기회',
 '와',
 '▁자원',
 '을',
 '▁공',
 '평',
 '하게',
 '▁분',
 '배',
 '해야',
 '▁하고',
 ',',
 '▁경쟁',
 '질서',
 '의',
 '▁공정',
 '성을',
 '▁회복',
 '해야',
 '▁한다고',
 '▁말했다',
 '[SEP]',
 '▁이',
 '▁시장은',
 '▁이날',
 '▁경',
 '향',
 '신문',
 '과',
 '의',
 '▁인터뷰에서',
 '▁이',
 '▁같이',
 '▁말',
 '하면서',
 '▁가장',
 '▁좋은',
 '▁방법',
 '은',
 '▁주',
 '▁',
 '방위',
 '군',
 '을',
 '▁',
 '시켜',
 '▁노조',
 '▁파',
 '업',
 '을',
 '▁지원',
 '한',
 '▁루',
 '즈',
 '벨트',
 '▁대통령',
 '처럼',
 '▁노조',
 '를',
 '▁강화',
 '하는',
 '▁것이라고',
 '▁덧붙였다',
 '[SEP]',
 '[SEP]',
 '▁이',
 '▁시장은',
 '▁또',
 '▁이',
 '▁사회',
 '▁기',
 '득',
 '권',
 '자는',
 '▁재벌',
 '로',
 '▁불리는',
 '▁경제',
 '권',
 '력',
 '이다',
 '[SEP]',
 '▁이',
 '▁거',
 '대한',
 '▁기',
 '득',
 '권',
 